<div style='background-image: url("baku.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.9) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">8th Munich Earth Skience School</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Sudelfeld, 4-9 March 2018</div>
        </div>
    </div>


<p style="width:50%;float:right;padding-left:50px">
<img src=./mess.jpg>
<span style="font-size:smaller">
</span>
</p>

# 6-C polarization analysis using point measurements of translational and rotational ground-motion
## Exercise 2 interfering P-waves

### Authors:
* David Sollberger (ETH Zurich)
* Cedric Schmelzbach (ETH Zurich)
* Cederic van Renterghem (ETH Zurich)
* Taufiqurrahman (LMU Munich)

***

In this notebook, the 6-C algorithm allows the retrieval of the directions of multiple, interfering events. The result is given for two interfering P-waves with parameters $m_1^P = (θ_P = 20 ◦ , φ = 20 ◦ , α = 300 \ m/s, β = 173 \ m/s)^T$ , and $m_2^P = (θ_P = 14 ◦ , φ = 0 ◦ ,α = 300 \ m/s , β = 173 \ m/s )^T$. The directions of both events are accurately obtained, as are the local P- and S-wave velocities (see Sollberger et.al.(2018) for details).

In [ ]:
# import libraries 
# ----------------
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

from ricker     import *
from awgn       import *
from polvect6C  import *
from music6C    import *

class structtype():
    pass

In [ ]:
# MUSIC parameters 
# ----------------
test_param       = structtype()
test_param.vp    = np.arange(200,500+50,50) # parameter space for vp
test_param.vs    = np.arange(100,200+5,5)   # parameter space for vs
test_param.theta = np.arange(1,40+1,1)      # parameter space for theta
test_param.phi   = np.arange(-40,50+1,1)    # parameter space for phi
wl = 0.6                                    # window length in seconds

# wave parameters
# wave 1
theta  = 20.                      # incidence angle [degree]
phi    = 20.                      # azimuth [degree]
vp     = 300.                     # P-wave velocity at the receiver [m/s]
vs     = 170.                     # S-wave velocity at the receiver [m/s]
v_scal = 1.2 * vs                 # scaling velocity (slightly overestimated S-wave velocity)

# wave 2
theta2 = 14.   
phi2   = 0.      

# seismogram parameters
tmin = 0.                         # minimum time of the seismogram [s]
tmax = 1.5                        # maximum time of the seismogram [s]
dt   = 0.25e-3                    # sampling interval [s]
t    = np.arange(tmin,tmax+dt,dt) # time [s]
t    = np.round(t,6)
t_1  = 0.75                       # arrival time of the wave 1 [s]
t_2  = 0.83                       # arrival time of the wave 2 [s]
fc   = 5.                         # dominant frequency [Hz]
wav  = ricker(dt,fc,tmax)         # Ricker wavelet

# generate synthetics
param        = structtype()
param.theta  = theta
param.phi    = phi
param.vp     = vp
param.vs     = vs

param2       = structtype()
param2.theta = theta2
param2.phi   = phi2
param2.vp    = vp
param2.vs    = vs

In [ ]:
# 6-C polarization vector 
# -----------------------
v  = polvect6C(param, v_scal,'P') # calculate the polarization vector for the given wave
v2 = polvect6C(param2,v_scal,'P') # calculate the polarization vector for the given wave
v  = v  / np.sqrt(sum(v **2))     # convert to unit vector
v2 = v2 / np.sqrt(sum(v2**2))     # convert to unit vector

data  = (np.zeros((len(t),6)))    # zero data initialization
indx  = t.tolist().index(t_1)     # index of arrival time of the wave 
indx2 = t.tolist().index(t_2)     # index of arrival time of the wave 
data[indx ,:] = v
data[indx2,:] = v2

for i in range(0, data[0,:].size - 1):
    data[:,i] = np.convolve(data[:,i], wav, 'same')
    
# add random noise 
SNR  = 50.                        # signal to noise ratio
data = awgn(data,SNR)

In [ ]:
# 6-C MUSIC 
# ---------
W = (wl / dt) # window length in samples
L = music6C(data,test_param,'P',v_scal,indx,W,'auto',1e-9,0.01)

In [ ]:
# remove instabilities 
# --------------------
L_ = np.nan_to_num(L)
L_[L_ == 0.] = 1.

# determine maximum value and index of the MUSIC estimator 
maxval  =    np.max(abs(L_[:]))
maxidx  = np.argmax(abs(L_[:]))
i,j,k,l = np.unravel_index(maxidx, L_[:].shape) # column- to row-major indexing

phitheta  = np.squeeze((abs(L_[:,:,k,l]))) # plane through best-fitting velocity parameters
vpvs      = np.squeeze((abs(L_[i,j,:,:]))) # plane through best-fitting arrival angles

In [ ]:
# plot configuration 
# ------------------
plt.figure(figsize=(8, 4))

plt.subplot(1,2,1)
vm1 = np.percentile((abs(phitheta)), 100)
plt.imshow((abs(phitheta)), cmap="jet", vmin=np.min((abs(phitheta))), vmax=vm1, aspect='auto', extent=[test_param.phi[0],
                                                                                                       test_param.phi[-1], 
                                                                                                       test_param.theta[-1], 
                                                                                                       test_param.theta[0]])
plt.xlabel(r'$\phi$ (°)')
plt.ylabel(r'$\theta$ (°)')
plt.plot(phi,theta,'r.')
plt.plot(phi2,theta2,'r.')

plt.subplot(1,2,2)
vm2 = np.percentile((abs(vpvs)), 100)
plt.imshow(abs(vpvs), cmap="jet", vmin=np.min((abs(vpvs))), vmax=vm2, aspect='auto', extent=[test_param.vs[0],
                                                                                             test_param.vs[-1], 
                                                                                             test_param.vp[-1], 
                                                                                             test_param.vp[0]])
plt.xlabel(r'$v_S$ (m/s)')
plt.ylabel(r'$v_P$ (m/s)')
plt.plot(vs,vp,'ro')

# show result on screen
plt.tight_layout()
plt.show()